In [1]:
import pandas as pd
import wget
import zipfile
import glob
import os
from bs4 import BeautifulSoup
import requests
from requests import get

In [2]:
#specify page to download zipfiles
page = requests.get('https://www.bls.gov/cew/downloadable-data-files.htm') 


#convert html to .text
tml = page.text

# passing file to the BeautifulSoup constructor
soup = BeautifulSoup(page.text, "html.parser")




In [6]:
#specify which years of data to download
years_url = [str(i) for i in range(2012, 2022)]
print(years_url)


['2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']


In [ ]:
#create list of downloaded zipfiles
link_list=[]


In [ ]:
# scraping naics files by year and excluding sic, download all relevant files for all years

# find every a element that has href attribute
for link in soup.find_all('a'):
    url = link.get('href')
    for year_url in years_url:
        # filter for last 10 years and annual by industry
        if (year_url in url) and ("annual_by_industry" in url): 
            print(url)
            file_name = url.split("csv/", 1)[1]
            link_list.append(file_name)
            with open (file_name, "wb") as file:
                response = get(url)
                file.write(response.content)
        else: 
            continue


In [12]:
# create a list that holds all dataframes combined for all years
li_all_year = []

# create a list that holds all dataframes combined for all years
#print(link_zipfile)
for link_zipfile in link_list:
    # create object for single year zipfile
    zf = zipfile.ZipFile(link_zipfile) 
    # return all files within single year's zipfile
    all_files = zf.namelist()
    # create list that holds all the dataframes combined for within that specific year 
    li_year = []
    #iteration for all files for a single year
    for csv_filename in all_files:
        
        if ".csv" in csv_filename:
            df_year = pd.read_csv(zf.open(csv_filename), 
            dtype={"area_fips": object},
            usecols=['area_fips' , 'industry_code', 'area_title', 'own_title', 'industry_title', 'total_annual_wages', 'annual_avg_emplvl', 'year'])
            li_year.append(df_year)
    concat_annual_df = pd.concat(li_year)
    li_all_year.append(concat_annual_df)

master_df = pd.concat(li_all_year)
    

In [29]:
# write df to csv
master_df.to_csv('master.csv')

In [ ]:
#read master.csv to test it was created
saved_file = pd.read_csv(open('master.csv'),
dtype={"year": int})


In [32]:
#see slice of the saved master.csv
print(saved_file[28500:28505])


       Unnamed: 0 area_fips industry_code  year  \
28500        2606     45000          1011  2021   
28501        2607     45001          1011  2021   
28502        2608     45003          1011  2021   
28503        2609     45005          1011  2021   
28504        2610     45007          1011  2021   

                             area_title own_title  \
28500       South Carolina -- Statewide   Private   
28501  Abbeville County, South Carolina   Private   
28502      Aiken County, South Carolina   Private   
28503  Allendale County, South Carolina   Private   
28504   Anderson County, South Carolina   Private   

                          industry_title  annual_avg_emplvl  \
28500  1011 Natural resources and mining              12250   
28501  1011 Natural resources and mining                 41   
28502  1011 Natural resources and mining                367   
28503  1011 Natural resources and mining                127   
28504  1011 Natural resources and mining                334

In [24]:
#see slice of master_df
print(master_df[28500:28505])


     area_fips industry_code  year                        area_title  \
2606     45000          1011  2021       South Carolina -- Statewide   
2607     45001          1011  2021  Abbeville County, South Carolina   
2608     45003          1011  2021      Aiken County, South Carolina   
2609     45005          1011  2021  Allendale County, South Carolina   
2610     45007          1011  2021   Anderson County, South Carolina   

     own_title                     industry_title  annual_avg_emplvl  \
2606   Private  1011 Natural resources and mining              12250   
2607   Private  1011 Natural resources and mining                 41   
2608   Private  1011 Natural resources and mining                367   
2609   Private  1011 Natural resources and mining                127   
2610   Private  1011 Natural resources and mining                334   

      total_annual_wages  
2606           587312557  
2607             1465937  
2608            18108667  
2609             5212722  

Previous working cell used to develop the code is below:

In [8]:
######### Begin Single Year, Single File


qcew = "https://data.bls.gov/cew/data/files/2021/csv/2021_annual_by_industry.zip" 
# Begin single year
wget.download(qcew) 
#download single year zip file
zf = zipfile.ZipFile("2021_annual_by_industry.zip") 
#create object for single year zipfile
df = pd.read_csv(zf.open('2021.annual.by_industry/2021.annual 238991 NAICS 238991 All other residential trade contractors.csv'), 
usecols=['area_fips' , 'industry_code', 'area_title', 'own_title', 'industry_title', 'total_annual_wages', 'annual_avg_emplvl', 'year'])
#read single year/ single industry .csv
df.head()
#verifty first lines of df


'2021_annual_by_industry.zip'

In [2]:
######### Begin Single Year, All Files


zf = zipfile.ZipFile("2021_annual_by_industry.zip") 
#create object for single year zipfile
all_files = zf.namelist()
li_2021 =[]
# create list for all dataframes for year
for filename in all_files:
    if ".csv" in filename:
        df_2021 = pd.read_csv(zf.open(filename),
        dtype={"area_fips": object},
        usecols=['area_fips' , 'industry_code', 'area_title', 'own_title', 'industry_title', 'total_annual_wages', 'annual_avg_emplvl', 'year'])
        li_2021.append(df_2021)
            #iteration for all files in a single year
    else: 
        continue

concat_2021_df = pd.concat(li_2021)

In [ ]:
############## previous working cells below 


In [ ]:
parent_dir = os.getcwd()
# parent directory path

directory = "documents/bls_data_project"
# directory

newpath = os.path.join(parent_dir, directory)
print(newpath)
# path

if os.path.exists(newpath):
    os.makedirs(newpath)

    # check if path exists, if false create new directory

In [13]:
len(li_2021)

4462

In [31]:
df_2021.dtypes


area_fips             object
industry_code          int64
year                   int64
area_title            object
own_title             object
industry_title        object
annual_avg_emplvl      int64
total_annual_wages     int64
dtype: object

In [3]:
all_files[0]


'2021.annual.by_industry/2021.annual 238991 NAICS 238991 All other residential trade contractors.csv'

In [41]:
len(df_2021)

2519

In [42]:
df.head()

,area_fips,industry_code,year,area_title,own_title,industry_title,annual_avg_emplvl,total_annual_wages
0,01000,238991,2021,Alabama -- Statewide,Private,NAICS 238991 All other residential trade contr...,1249,60412058
1,01001,238991,2021,"Autauga County, Alabama",Private,NAICS 238991 All other residential trade contr...,0,0
2,01003,238991,2021,"Baldwin County, Alabama",Private,NAICS 238991 All other residential trade contr...,128,5836986
3,01005,238991,2021,"Barbour County, Alabama",Private,NAICS 238991 All other residential trade contr...,0,0
4,01009,238991,2021,"Blount County, Alabama",Private,NAICS 238991 All other residential trade contr...,0,0


In [4]:
df = pd.read_csv(all_files[0])

In [7]:
df.dtypes
df[col_list].head()

,area_fips,area_title,own_title,industry_title,total_annual_wages,annual_avg_emplvl,year
0,01000,Alabama -- Statewide,Private,NAICS 238991 All other residential trade contr...,60412058,1249,2021
1,01001,"Autauga County, Alabama",Private,NAICS 238991 All other residential trade contr...,0,0,2021
2,01003,"Baldwin County, Alabama",Private,NAICS 238991 All other residential trade contr...,5836986,128,2021
3,01005,"Barbour County, Alabama",Private,NAICS 238991 All other residential trade contr...,0,0,2021
4,01009,"Blount County, Alabama",Private,NAICS 238991 All other residential trade contr...,0,0,2021


In [8]:
df.columns

Index(['area_fips', 'own_code', 'industry_code', 'agglvl_code', 'size_code',
       'year', 'qtr', 'disclosure_code', 'area_title', 'own_title',
       'industry_title', 'agglvl_title', 'size_title',
       'annual_avg_estabs_count', 'annual_avg_emplvl', 'total_annual_wages',
       'taxable_annual_wages', 'annual_contributions', 'annual_avg_wkly_wage',
       'avg_annual_pay', 'lq_disclosure_code', 'lq_annual_avg_estabs_count',
       'lq_annual_avg_emplvl', 'lq_total_annual_wages',
       'lq_taxable_annual_wages', 'lq_annual_contributions',
       'lq_annual_avg_wkly_wage', 'lq_avg_annual_pay', 'oty_disclosure_code',
       'oty_annual_avg_estabs_count_chg',
       'oty_annual_avg_estabs_count_pct_chg', 'oty_annual_avg_emplvl_chg',
       'oty_annual_avg_emplvl_pct_chg', 'oty_total_annual_wages_chg',
       'oty_total_annual_wages_pct_chg', 'oty_taxable_annual_wages_chg',
       'oty_taxable_annual_wages_pct_chg', 'oty_annual_contributions_chg',
       'oty_annual_contributions_pct_

In [9]:
df.industry_code

0       238991
1       238991
2       238991
3       238991
4       238991
         ...  
2766    238991
2767    238991
2768    238991
2769    238991
2770    238991
Name: industry_code, Length: 2771, dtype: int64

In [29]:
df_list = [pd.read_csv(f, dtype={'area_fips': str}) for f in all_files]

In [30]:
twentyone = pd.concat(df_list, ignore_index=True)


In [ ]:
national = twentyone['area_title'].str.contains('statewide', case=False, na=False)

In [ ]:
twentyone = twentyone[national]

In [ ]:
twentyone.head(3)

In [6]:
col_list = ['area_fips' , 'area_title', 'own_title', 'industry_title', 'total_annual_wages', 'annual_avg_emplvl', 'year']

In [118]:
twentyone = twentyone[col_list]

In [119]:
twentyone[national]

,area_fips,area_title,own_title,industry_title,total_annual_wages,annual_avg_emplvl,year
0,01000,Alabama -- Statewide,Private,NAICS 238991 All other residential trade contr...,60297403,1246,1246
47,02000,Alaska -- Statewide,Private,NAICS 238991 All other residential trade contr...,7911995,129,129
56,04000,Arizona -- Statewide,Private,NAICS 238991 All other residential trade contr...,247459724,4342,4342
72,05000,Arkansas -- Statewide,Private,NAICS 238991 All other residential trade contr...,46546916,1102,1102
113,06000,California -- Statewide,Private,NAICS 238991 All other residential trade contr...,1347624201,21586,21586
...,...,...,...,...,...,...,...
3626681,55000,Wisconsin -- Statewide,Local Government,NAICS 561990 All other support services,0,0,0
3626682,55000,Wisconsin -- Statewide,Private,NAICS 561990 All other support services,134879886,3995,3995
3626742,56000,Wyoming -- Statewide,Private,NAICS 561990 All other support services,17297582,458,458
3626757,72000,Puerto Rico -- Statewide,Private,NAICS 561990 All other support services,8483082,343,343
